In [1]:
#Se conecta Google Colaboratory con Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Instala pyspark en Google Colaboratory
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488493 sha256=eecaa6a96d60fcbb496de850d65c6aa4bb0c4045f709a3cadb1e1c91c7427d2c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
import warnings
warnings.filterwarnings("ignore") # Se utiliza para gestionar las advertencias y mantener el código limpio.

In [4]:
import os # Proporciona funciones para interactuar con el sistema operativo.
import requests # Se utiliza para realizar solicitudes HTTP.
import pandas as pd # Una librería de análisis de datos.
import seaborn as sns #S e utiliza para la visualización de datos.
import pyspark.pandas as ps # Proporciona una interfaz para trabajar con datos en Spark utilizando el formato de DataFrame de pandas.
import json # Se utiliza para trabajar con datos en formato JSON.
from pyspark.sql import SparkSession # Se utiliza para crear una instancia de SparkSession, que es la entrada principal para trabajar con Spark SQL.
from pyspark.sql import functions as F #  Proporciona funciones para trabajar con datos en Spark DataFrame.
from pyspark.sql.functions import array_contains # Esta función se utiliza para filtrar los datos basados en la presencia de un valor en un array.
from pyspark.sql.functions import sum, col # Se utiliza para acceder a una columna en un DataFrame de Spark.
from pyspark.sql.functions import split, substring, concat_ws
from pyspark.sql.functions import expr

In [5]:
# Crear una sesión de Spark
spark = SparkSession.builder\
        .master("local")\
        .appName("ETL-review")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [6]:
#Se lee el archivo review de yelp

df = spark.read.json("/content/drive/MyDrive/Colab-Notebooks/Yelp/review.json")

In [7]:
df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|  3.0|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|  5.0|Wow!  Yummy, diff...|     1|_7bHUi9Uuf5__HHc_...|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|  4.0|Cute inter

In [8]:
df = df.drop('date')


In [9]:
# Cambia el nombre de la columna 'stars' por 'rating'
df = df.withColumnRenamed('stars', 'rating')

In [10]:
# Cambia el nombre de la columna 'text' por 'review'
df = df.withColumnRenamed('text', 'review')

In [11]:
# Reordena las columnas
df = df.select('review_id', 'user_id', 'business_id', 'review', 'rating', 'cool', 'useful', 'funny')


In [12]:

df.show()

+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|           review_id|             user_id|         business_id|              review|rating|cool|useful|funny|
+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|If you decide to ...|   3.0|   0|     0|    0|
|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|I've taken a lot ...|   5.0|   1|     1|    0|
|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|Family diner. Had...|   3.0|   0|     0|    0|
|AqPFMleE6RsU23_au...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|Wow!  Yummy, diff...|   5.0|   1|     1|    0|
|Sx8TMOWLNuJBWer-0...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|Cute interior and...|   4.0|   1|     1|    0|
|JrIxlS1TzJ-iCu79u...|eUta8W_HdHMXPzLBB...|04UD14gamNjLY0IDY...|I am a long term ...|   1.0|   1|     1|    2|
|

In [13]:
from pyspark.sql.functions import col, regexp_replace

In [14]:
# Reemplazar las comas en la columna 'text'
df = df.withColumn('review', regexp_replace(col('review'), ',', ''))

# Mostrar el DataFrame final
df.show()

+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|           review_id|             user_id|         business_id|              review|rating|cool|useful|funny|
+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+
|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|If you decide to ...|   3.0|   0|     0|    0|
|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|I've taken a lot ...|   5.0|   1|     1|    0|
|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|Family diner. Had...|   3.0|   0|     0|    0|
|AqPFMleE6RsU23_au...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|Wow!  Yummy diffe...|   5.0|   1|     1|    0|
|Sx8TMOWLNuJBWer-0...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|Cute interior and...|   4.0|   1|     1|    0|
|JrIxlS1TzJ-iCu79u...|eUta8W_HdHMXPzLBB...|04UD14gamNjLY0IDY...|I am a long term ...|   1.0|   1|     1|    2|
|

In [15]:
from pyspark.sql.functions import col

# Verificar si la columna 'funny' contiene solo valores numéricos
has_only_numeric_values = df.filter(col("funny").cast("float").isNotNull()).count() == df.count()

if has_only_numeric_values:
    print("La columna 'funny' contiene solo valores numéricos.")
else:
    print("La columna 'funny' contiene valores que no son numéricos.")

La columna 'funny' contiene solo valores numéricos.


In [16]:
from pyspark.sql.functions import col, countDistinct
from pyspark.sql.functions import col, when

In [17]:
# Convertir la columna 'rating' a tipo float, reemplazando los valores no numéricos por 0
df = df.withColumn("rating", when(col("rating").cast("float").isNotNull(), col("rating").cast("float")).otherwise(0.0))

# Verificar si la columna 'rating' contiene solo valores numéricos
has_only_numeric_values = df.filter(col("rating").cast("float").isNotNull()).count() == df.count()

if has_only_numeric_values:
    print("La columna 'rating' ahora contiene solo valores numéricos.")
else:
    print("La columna 'rating' contiene valores que no son numéricos.")

La columna 'rating' ahora contiene solo valores numéricos.


In [18]:
# Cuenta el número de nulos en cada columna

def conteo_nulos(dataframe):
  conteo_nulos_por_columna = dataframe.agg(*[sum(col(c).isNull().cast("int")).alias(c) for c in dataframe.columns])

  # Muestra el resultado
  conteo_nulos_por_columna.show()

In [19]:
conteo_nulos(df)

+---------+-------+-----------+------+------+----+------+-----+
|review_id|user_id|business_id|review|rating|cool|useful|funny|
+---------+-------+-----------+------+------+----+------+-----+
|        0|      0|          0|     0|     0|   0|     0|    0|
+---------+-------+-----------+------+------+----+------+-----+



In [20]:
df.count()

6990280

In [21]:
# Eliminar duplicados basándote en todas las columnas
df_no_duplicates = df.dropDuplicates()

# Contar las filas después de eliminar duplicados
print("Número de filas después de eliminar duplicados:", df_no_duplicates.count())

Número de filas después de eliminar duplicados: 6990280


In [22]:

df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- review: string (nullable = true)
 |-- rating: double (nullable = true)
 |-- cool: long (nullable = true)
 |-- useful: long (nullable = true)
 |-- funny: long (nullable = true)



In [24]:
# Reemplazar las comillas en todas las columnas
df_no_quotes = df.select([
    regexp_replace(col(column), '"', '').alias(column)
    for column in df_no_duplicates.columns
])

In [25]:
# Reemplazar las comas en todas las columnas
df_no_commas = df_no_quotes.select([
    regexp_replace(col(column), ',', '').alias(column)
    for column in df_no_duplicates.columns
])

In [26]:
df = df_no_commas

In [27]:
# Convertir la columna "rating" a tipo entero
df = df.withColumn("rating", col("rating").cast("float"))

In [28]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from textblob import TextBlob

# Función para determinar el sentimiento basado en el texto y la calificación
def get_sentiment(text, rating):
    if text:
        analysis = TextBlob(text)
        if analysis.sentiment.polarity > 0:
            return 1  # Sentimiento positivo
        elif analysis.sentiment.polarity < 0:
            return 2  # Sentimiento negativo
        else:
            return 0  # Sentimiento neutro
    else:
        if rating is not None:
            if rating >= 4:
                return 1  # Calificación alta considerada como sentimiento positivo
            elif rating <= 2:
                return 2  # Calificación baja considerada como sentimiento negativo
            else:
                return 0  # Calificación intermedia considerada como sentimiento neutro
        else:
            return 0  # Si tanto el texto como la calificación están vacíos, asignar 0

# Registrar la función como UDF
sentiment_udf = udf(lambda text, rating: get_sentiment(text, rating), IntegerType())

In [29]:
# Aplicar la función a la columna "review" y crear una nueva columna "sentiment"
df_with_sentiment = df.withColumn("sentiment", sentiment_udf("review", "rating"))

In [30]:
# Mostrar el DataFrame final
df_with_sentiment.show()

+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+---------+
|           review_id|             user_id|         business_id|              review|rating|cool|useful|funny|sentiment|
+--------------------+--------------------+--------------------+--------------------+------+----+------+-----+---------+
|KU_O5udG6zpxOg-Vc...|mh_-eMZ6K5RLWhZyI...|XQfwVwDr-v0ZS3_Cb...|If you decide to ...|   3.0|   0|     0|    0|        1|
|BiTunyQ73aT9WBnpR...|OyoGAe7OKpv6SyGZT...|7ATYjTIgM3jUlt4UM...|I've taken a lot ...|   5.0|   1|     1|    0|        1|
|saUsX_uimxRlCVr67...|8g_iMtfSiwikVnbP2...|YjUWPpI6HXG530lwP...|Family diner. Had...|   3.0|   0|     0|    0|        1|
|AqPFMleE6RsU23_au...|_7bHUi9Uuf5__HHc_...|kxX2SOes4o-D3ZQBk...|Wow!  Yummy diffe...|   5.0|   1|     1|    0|        1|
|Sx8TMOWLNuJBWer-0...|bcjbaE6dDog4jkNY9...|e4Vwtrqf-wpJfwesg...|Cute interior and...|   4.0|   1|     1|    0|        1|
|JrIxlS1TzJ-iCu79u...|eUta8W_HdH

In [31]:
df_with_sentiment.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- review: string (nullable = true)
 |-- rating: float (nullable = true)
 |-- cool: string (nullable = true)
 |-- useful: string (nullable = true)
 |-- funny: string (nullable = true)
 |-- sentiment: integer (nullable = true)



In [ ]:
conteo_nulos(df_with_sentiment)

In [23]:
df_repartitioned = df.repartition(20)  # 20 particiones para obtener 20 archivos CSV

CARGA DE LOS ARCHIVOS

In [32]:
# Ruta al archivo CSV local
file_path = '/content/drive/MyDrive/Colab-Notebooks/transformaciones/review-limpio.csv'

# Escribe el DataFrame a un solo archivo CSV localmente con separador ","
df_with_sentiment.coalesce(5).write.csv(file_path, header=True, sep=',')

AnalysisException: [PATH_ALREADY_EXISTS] Path file:/content/drive/MyDrive/Colab-Notebooks/Datos-Limpios/review-limpiowsen22.csv already exists. Set mode as "overwrite" to overwrite the existing path.